In [1]:
import numpy as np
import pandas as pd
import torch
from tqdm import tqdm

In [2]:
seqNameDF = pd.read_csv("aaSeqCdr3Combined")

In [3]:
samplenames = seqNameDF.sampleName.unique()

In [4]:
cdr3seqs = seqNameDF.amino_acid.unique()

In [5]:
samplenames_dict = {samplenames[i]: i for i in range(len(samplenames))}

In [6]:
cdr3_dict = {cdr3seqs[i]: i for i in range(len(cdr3seqs))}

In [ ]:
#print(len(seqNameDF))

In [8]:
rowidxs = np.zeros(len(seqNameDF))
colidxs = np.zeros(len(seqNameDF))

In [9]:
# get dict number from cdr3 string:
def cdr3_tonum(cdr3):
    numcdr3 = cdr3_dict[cdr3]
    return numcdr3

In [10]:
# get dict number from sample string:
def sample_tonum(sample):
    numsample = samplenames_dict[sample]
    return numsample

In [11]:
rowidxs = np.vectorize(cdr3_tonum)(seqNameDF["amino_acid"])

In [12]:
colidxs = np.vectorize(sample_tonum)(seqNameDF["sampleName"])

In [13]:
#v = np.ones(len(seqNameDF))

In [14]:
ii = np.concatenate((rowidxs, colidxs), axis=0)

In [15]:
ii = ii.reshape((2,len(seqNameDF)))
del seqNameDF

In [16]:
sharing = list()

In [ ]:
for nusmpl in tqdm(range(2,len(samplenames))):
    # choose a random subset of nosmpl samples:
    #nusmpl = 4
    sample_idxs = np.random.choice(len(samplenames), nusmpl, replace=False)
    #print(sample_idxs)
    #reduce ii to include only these samples:
    aa = ii[1,:]
    bb = np.in1d(aa, sample_idxs).nonzero()[0]
    ii_reduced = ii[:,bb]
    del aa, bb
    #sort ii_reduced:
    ii_reduced = ii_reduced[:,np.argsort(ii_reduced[0,:])].T
    ii_reduced = ii_reduced[:,0]
    ii_sorted_df = pd.DataFrame(ii_reduced, columns=["zero"])
    del ii_reduced
    df2 = ii_sorted_df.groupby('zero').size()#.reset_index(name='count')
    aa = df2>1
    del df2
    df2_count = aa.sum()
    print(df2_count)
    sharing.append(df2_count)

In [18]:
#s = torch.sparse_coo_tensor(ii , v, device=torch.device('cuda:0'))

In [19]:
# import pickle

# with open('savedenvironment.pickle', 'wb') as handle:
#     pickle.dump([cdr3_dict, samplenames_dict, s], handle, protocol=pickle.HIGHEST_PROTOCOL)